In [4]:
import spacy
from sklearn.metrics import classification_report

def predict_entities(text):
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    return [(ent.text, ent.label_) for ent in doc.ents]

def evaluate_ner(text_and_labels):
    true_labels = []
    pred_labels = []
    for text, true_ents in text_and_labels:
        pred_ents = predict_entities(text)
        true_labels.extend([label for _, label in true_ents])
        if len(pred_ents) < len(true_ents):
            pred_labels.extend([label for _, label in pred_ents] + ["O"] * (len(true_ents) - len(pred_ents)))  # Pad with "O"
        else:
            pred_labels.extend([label for _, label in pred_ents[:len(true_ents)]])  # Truncate to match length
    return classification_report(true_labels, pred_labels, zero_division=0)

text = "Apple CEO Tim Cook is looking at buying U.K. startup for $1 billion"
print("\nExample text entities:")
for entity, label in predict_entities(text):
    print(f"{entity} ({label})")

test_data = [
    ("Apple is looking at buying U.K. startup for $1 billion", [("Apple", "ORG"), ("U.K.", "GPE"), ("$1 billion", "MONEY")]),
    ("San Francisco considers banning sidewalk delivery robots", [("San Francisco", "GPE"), ("sidewalk delivery robots", "MISC")]),
    ("London is a big city in the U.K.", [("London", "GPE"), ("U.K.", "GPE")]),
    ("Google is a technology company", [("Google", "ORG")]),
    ("Microsoft is acquiring LinkedIn for $26.2 billion", [("Microsoft", "ORG"), ("LinkedIn", "ORG"), ("$26.2 billion", "MONEY")]),
]
print("\nEvaluation report:")
print(evaluate_ner(test_data))


Example text entities:
Apple (ORG)
Tim Cook (PERSON)
U.K. (GPE)
$1 billion (MONEY)

Evaluation report:
              precision    recall  f1-score   support

         GPE       1.00      1.00      1.00         4
        MISC       0.00      0.00      0.00         1
       MONEY       1.00      1.00      1.00         2
           O       0.00      0.00      0.00         0
         ORG       1.00      1.00      1.00         4

    accuracy                           0.91        11
   macro avg       0.60      0.60      0.60        11
weighted avg       0.91      0.91      0.91        11

